In [ ]:
from IPython.core.display import HTML

In [ ]:
display(HTML("<style>.container {width:95% !important; }</stype>"))

https://voice.mozilla.org/en/datasets

### Content
Total hours: ? h.

Language: Spanish.

Number of speakers: ???.

Type of speech: ??? # Clean speech.


#### Collection methodology

????

### Extra
Collected by: 

### License
License : Public Domain

### Downloading 
#### Method 


In [ ]:
import glob
import os
#import subprocess
#import tarfile
#import wget

In [ ]:
#%matplotlib inline
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import IPython.display as ipd
from ipywidgets import interact
import ipywidgets as widgets
import re

In [ ]:
import soundfile as sf
#import IPython.display as ipd
#from ipywidgets import interact
#import ipywidgets as widgets
from pathlib import Path


In [ ]:
%ls

In [ ]:
#%ls asr-spanish-v1-carlfm01\audios\0000df16-47ea-428f-8367-df2ce365d5c4.wav

In [ ]:
#%ls asr-spanish-v1-carlfm01\asr-spanish-v1-carlfm01\audios\0000df16-47ea-428f-8367-df2ce365d5c4.wav

In [ ]:
pwd

In [ ]:
egg_folder_name = os.path.join('/gong-asr/kaldi-12/Domain-specific-ESPnet','espnet','egs',
                        'spanish_common_voice','asr1')

data_folder_name = os.path.join(egg_folder_name, 'raw_data', 'CommonVoiceSpanish', 'decompressed')

kaldi_ready_data_folder_name = os.path.join(egg_folder_name, 'data')

In [ ]:
! ls -l {data_folder_name} | wc -l

In [ ]:
description_file_name = os.path.join(data_folder_name,'train.tsv')

In [ ]:
%ls -l {description_file_name}

In [ ]:
df = pd.read_csv(description_file_name, sep='\\t')

In [ ]:
df.head()

In [ ]:
len(df.index) == len(set(df.index))

In [ ]:
df_raw = df.loc[df.index[0]]
df_raw

In [ ]:
def segment_by_df_raw(df_raw):
    audio, sample_rate = librosa.load(os.path.join(data_folder_name,'clips',df_raw.path))
    plt.rcParams['figure.figsize'] = (15,7)
    plt.title(f'Waveform of Audio Example: {df_raw["path"]}')
    plt.ylabel('Amplitude')

    print(df_raw['sentence'])
    print(f'sample_rate: {sample_rate}')
    _ = librosa.display.waveplot(audio)
    return ipd.Audio(audio, rate=sample_rate)

In [ ]:
segment_by_df_raw(df_raw)

In [ ]:
def segment_by_idx(idx):
    return segment_by_df_raw(df.iloc[idx])

In [ ]:
interact(segment_by_idx, idx=widgets.IntSlider(min=0, max=df.shape[0]-1, step=1, value=0));

In [ ]:
#peth = '/espnet/egs/spanish_common_voice/asr1/raw_data/CommonVoiceSpanish/decompressed/clips/common_voice_es_19499893.mp3'
file_name = 'common_voice_es_19499893.mp3'
def show_audio(file_name):
    try:
        audio, sample_rate = librosa.load(os.path.join(data_folder_name,'clips',file_name))
        plt.rcParams['figure.figsize'] = (15,7)
    #     plt.title(f'Waveform of Audio Example: {df_raw["path"]}')
        plt.ylabel('Amplitude')
        _ = librosa.display.waveplot(audio)
        res = ipd.Audio(audio, rate=sample_rate)    
    except:
        print("An exception occurred")
        res = None
    return res

In [ ]:
show_audio(file_name)

In [ ]:
train_test = 'test'
kaldi_ready_md_folder = os.path.join(kaldi_ready_data_folder_name, 'test')
kaldi_ready_audio_folder = os.path.join(kaldi_ready_data_folder_name, 'test')

In [ ]:
! ls -l {kaldi_ready_md_folder}

In [ ]:
! head -n5 {os.path.join(kaldi_ready_md_folder, 'wav.scp')}

In [ ]:
! head -n5 {os.path.join(kaldi_ready_md_folder, 'text')}

In [ ]:
wavscp_df = pd.read_csv(os.path.join(kaldi_ready_md_folder, 'wav.scp'), sep=' ',header=None)\
.set_index(0)
wavscp_df.columns = ['wav']
wavscp_df.head()

In [ ]:
with open(os.path.join(kaldi_ready_md_folder, 'text'), 'r') as text_fd:
    text_str = text_fd.read()
text_df = pd.DataFrame(np.array([(lambda x: (x[0], ' '.join(x[1:])))(ent.split(' ')) for ent in text_str.split('\n') if ''!=ent])).set_index(0)
text_df.columns = ['text']
text_df.head()

In [ ]:
set(text_df.index == wavscp_df.index)

In [ ]:
parsed_df = wavscp_df.join(text_df)
parsed_df.head()

In [ ]:
def segment_by_parsed_df_raw(df_raw):
    audio, sample_rate = sf.read(os.path.join(egg_folder_name, df_raw.wav))
    plt.rcParams['figure.figsize'] = (15,7)
    plt.title(f'Waveform of Audio Example: {df_raw["wav"]}')
    plt.ylabel('Amplitude')

    print(df_raw['text'])
    print(f'sample_rate: {sample_rate}')                            
    _ = librosa.display.waveplot(audio)
    return ipd.Audio(audio, rate=sample_rate)

In [ ]:
def parsed_segment_by_idx(idx):
    return segment_by_parsed_df_raw(parsed_df.iloc[idx])

In [ ]:
! ls -l /gong-asr/kaldi-12/Domain-specific-ESPnet/espnet/egs/spanish_common_voice/asr1/data/train/downloads/comvoice/common_voice_es_19698530.wav

In [ ]:
interact(parsed_segment_by_idx, idx=widgets.IntSlider(min=0, max=parsed_df.shape[0]-1, step=1, value=0));

## json labels

In [ ]:
train_json_local_path = 'dump/train_set/deltafalse/data_unigram5000.json'
train_dev_json_local_path = 'dump/train_dev/deltafalse/data_unigram5000.json'
test_json_local_path = 'dump/test/deltafalse/data_unigram5000.json'

In [ ]:
import json

In [ ]:
train_dict = json.loads(open(os.path.join(egg_folder_name, train_json_local_path),'r').read())
train_dev_dict = json.loads(open(os.path.join(egg_folder_name, train_dev_json_local_path),'r').read())
test_dict = json.loads(open(os.path.join(egg_folder_name, test_json_local_path),'r').read())

In [ ]:
print(len(list(train_dict['utts'].keys())))
print(len(list(train_dev_dict['utts'].keys())))
print(len(list(test_dict['utts'].keys())))

In [ ]:
list(train_dict['utts'].keys())[0]

In [ ]:
train_dict['utts'][list(train_dict['utts'].keys())[2]]

In [ ]:
train_dict['utts'][list(train_dict['utts'].keys())[0]]['utt2spk'] 

In [ ]:
train_spks = (lambda x: {x['utts'][k]['utt2spk'] for k in x['utts'].keys()})(train_dict)
len(train_spks)

In [ ]:
train_dev_spks = (lambda x: {x['utts'][k]['utt2spk'] for k in x['utts'].keys()})(train_dev_dict)
len(train_dev_spks)

In [ ]:
test_spks = (lambda x: {x['utts'][k]['utt2spk'] for k in x['utts'].keys()})(test_dict)
len(test_spks)

check there is no intersection of speekers

In [ ]:
print(len(np.intersect1d(train_spks, train_dev_spks)))
print(len(np.intersect1d(train_spks, test_spks)))
print(len(np.intersect1d(train_dev_spks, test_spks)))

In [ ]:
train_input_shapes = (lambda x: [x['utts'][k]['input'][0]['shape'] for k in x['utts'].keys()])(train_dict)
train_output_shapes = (lambda x: [x['utts'][k]['output'][0]['shape'] for k in x['utts'].keys()])(train_dict)
print(set(np.array(train_input_shapes).T[1])) # should be correspondent to 80 fbanks 
print(set(np.array(train_output_shapes).T[1])) # should be correspondent to 5000 subwords in the dict

In [ ]:
train_dev_input_shapes = (lambda x: [x['utts'][k]['input'][0]['shape'] for k in x['utts'].keys()])(train_dev_dict)
train_dev_output_shapes = (lambda x: [x['utts'][k]['output'][0]['shape'] for k in x['utts'].keys()])(train_dev_dict)
print(set(np.array(train_dev_input_shapes).T[1])) # should be correspondent to 80 fbanks 
print(set(np.array(train_dev_output_shapes).T[1])) # should be correspondent to 5000 subwords in the dict

In [ ]:
test_input_shapes = (lambda x: [x['utts'][k]['input'][0]['shape'] for k in x['utts'].keys()])(test_dict)
test_output_shapes = (lambda x: [x['utts'][k]['output'][0]['shape'] for k in x['utts'].keys()])(test_dict)
print(set(np.array(test_input_shapes).T[1])) # should be correspondent to 80 fbanks 
print(set(np.array(test_output_shapes).T[1])) # should be correspondent to 5000 subwords in the dict

In [ ]:
egg_folder_name

In [ ]:
import glob

In [ ]:
encoders_file_list = glob.glob(str(os.path.join(egg_folder_name, "exp/train_set_pytorch_train/results/att_ws/"))+'*encoder*')
decoders_file_list = glob.glob(str(os.path.join(egg_folder_name, "exp/train_set_pytorch_train/results/att_ws/"))+'*decoder*')

In [ ]:
encoders_file_list[0]

In [ ]:
# ! ls -l {os.path.join(egg_folder_name, "exp/train_set_pytorch_train/results/att_ws/")}

In [ ]:
len(encoders_file_list)

In [ ]:
len(decoders_file_list)

In [ ]:
from IPython.display import Image

# Image(file_list[1])

In [ ]:
def show_encoder_file_by_idx(idx):
    print(encoders_file_list[idx].split('/')[-1])
    return Image(encoders_file_list[idx])

def show_decoder_file_by_idx(idx):
    print(decoders_file_list[idx].split('/')[-1])
    return Image(decoders_file_list[idx])

In [ ]:
interact(show_encoder_file_by_idx, idx=widgets.IntSlider(min=0, max=len(encoders_file_list)-1, step=1, value=0));

In [ ]:
interact(show_decoder_file_by_idx, idx=widgets.IntSlider(min=0, max=len(decoders_file_list)-1, step=1, value=0));

In [ ]:
pwd

In [ ]:
with open(os.path.join(egg_folder_name,'exp/train_set_pytorch_train/train.log'), 'r') as rfd:
    train_log_str = rfd.read()

In [ ]:
print(train_log_str[:500])

In [ ]:
# ll /espnet/tools/venv/lib/python3.7/site-packages/librosa/util/decorators.py

In [ ]:
train_log_lines = [ent for ent in train_log_str.split('\n')]

In [ ]:
re.sub(' +',',',[ent for ent in train_log_str.split('\n') if 'iteration' in ent][0]).split(',')

In [ ]:
# [train_log_lines[idx] for idx, ent in enumerate(train_log_lines[2:]) if 15 == len(re.sub(' +',',',ent).split(','))]

In [ ]:
# [ent for ent in train_log_str.split('\n') if not 'length' in ent]

In [ ]:
# acoustic_learning_outputs = [ent for ent in train_log_str.split('\n') if 'epoch' in ent]
# acoustic_learning_outputs

In [ ]:
#[train_log_lines[idx+2] for idx, ent in enumerate(train_log_lines[2:]) if not '2020-06-28' in ent and '\x1b[J' in ent and not 'total' in ent]
[' '.join(re.sub(' +',',',[ent for ent in train_log_str.split('\n') if 'iteration' in ent][0]).split(',')[1:])] + \
[' '.join(re.sub(' +',',',ent).split(',')[1:]) for idx, ent in enumerate(train_log_lines[2:]) if not '2020-06-28' in ent and '\x1b[J' in ent and not 'total' in ent \
 and 15 == len(re.sub(' +',',',ent).split(',')[1:])]

In [ ]:
evaluation_stats_df = pd.DataFrame(np.array([re.sub(' +',',',ent).split(',')[1:-1] for idx, ent in enumerate(train_log_lines[2:]) if not '2020-06-28' in ent and '\x1b[J' in ent and not 'total' in ent \
 and 15 == len(re.sub(' +',',',ent).split(',')[1:])]), columns=re.sub(' +',',',[ent for ent in train_log_str.split('\n') if 'iteration' in ent][0]).split(',')[1:]).set_index('iteration')

In [ ]:
evaluation_stats_df

In [ ]:
evaluation_stats_df.drop(labels=['elapsed_time','main/acc','validation/main/acc', 'validation/main/cer', 'validation/main/wer', 'main/cer_ctc','validation/main/cer_ctc'],axis=1).applymap(float)\
.plot(title='"loss"',figsize=(15,5));

In [ ]:
evaluation_stats_df.drop(labels=['elapsed_time'],axis=1)[['main/acc','validation/main/acc']].applymap(float)\
.plot(title='"accuracy"',figsize=(15,5));

In [ ]:
evaluation_stats_df.drop(labels=['elapsed_time'],axis=1)[['validation/main/cer', 'main/cer_ctc','validation/main/cer_ctc']].applymap(float)\
.plot(title='"cer"',figsize=(15,5));

In [ ]:
evaluation_stats_df.drop(labels=['elapsed_time'],axis=1)[['validation/main/wer']].applymap(float)\
.plot(title='"validation/main/wer"',figsize=(15,5));

In [ ]:
[(train_log_lines[idx-1],re.sub(' +',',',ent).split(',')) for idx, ent in enumerate(train_log_lines) if 'validation/main/loss_att' in ent]

In [ ]:
# [(train_log_lines[idx-1],re.sub(' +',',',ent).split(',')) for idx, ent in enumerate(train_log_lines) if \
#  9 == len(re.sub(' +',',',ent).split(','))]

In [ ]:
re.sub(' +',',',[ent for ent in train_log_str.split('\n') if 'iteration' in ent][0]).split(',')[1:8]

In [ ]:
pd.DataFrame(np.array([re.sub(' +',',',ent).split(',')[1:-1] for idx, ent in enumerate(train_log_lines[2:]) if not '2020-06-28' in ent and '\x1b[J' in ent and not 'total' in ent \
 and 8 == len(re.sub(' +',',',ent).split(',')[1:])])#columns=re.sub(' +',',',[ent for ent in train_log_str.split('\n') if 'iteration' in ent][0]).split(',')[1:8]
            )

In [ ]:
[(train_log_lines[idx +1][len('\x1b[J'):], train_log_lines[idx +2], train_log_lines[idx +3], train_log_lines[idx +4]
#   , 
#   train_log_lines[idx +7], train_log_lines[idx +14]
 ) for idx, ent in enumerate(train_log_lines[2:]) if 'this epoch' in ent]

In [ ]:
# acoustic_learning_outputs[1]

In [ ]:
# enumerated_acoustic_learning_outputs = [(idx, ent) for (idx, ent) in enumerate(train_log_str.split('\n')) if 'epoch' in ent]

In [ ]:
# [(train_log_lines[ent[0]-1], ent[1], train_log_lines[ent[0]+1], train_log_lines[ent[0]+2]) for ent in enumerated_acoustic_learning_outputs]

In [ ]:
dir_file_list = glob.glob(str(os.path.join(egg_folder_name)+'/*'))

In [ ]:
egg_folder_name

In [ ]:
dir_file_list

In [ ]:
[ent for ent in dir_file_list if Path(ent).is_file()]

In [ ]:
dir_dirs_list = [ent1 for ent1 in [ent for ent in dir_file_list if Path(ent).is_dir()] if not ('downloads' in ent1 or 'raw_data' in ent1)]
dir_dirs_list

In [ ]:
target_path = Path(egg_folder_name)

In [ ]:
print(dir(target_path))

In [ ]:
# target_path = (lambda x: Path(str(os.path.join(list(x.parts[:-2])+[x.parts[-2]+'_backup1']))))(Path(egg_folder_name))
# target_path

### Backup the results

In [ ]:
target_path = (lambda x: Path(os.path.join(*tuple(list(x.parts[:-2])+[x.parts[-2]+'_backup1', x.parts[-1]]))))(Path(egg_folder_name))
target_path

In [ ]:
# target_path.mkdir(exist_ok=False)

In [ ]:
# for ent in dir_file_list:
#     if Path(ent).is_file():
#         ! cp {ent} {target_path}

In [ ]:
# for ent in dir_file_list:
#     if Path(ent).is_dir() and not ('downloads' in ent or 'raw_data' in ent):
#         ! cp -r {ent} {target_path}

In [ ]:
raw_files_list = glob.glob(str(os.path.join(data_folder_name,'clips')+"/*"))

In [ ]:
def timelingth_raw(file_name):
    audio, sample_rate = librosa.load(file_name)
    return (audio.shape[0]/sample_rate)
#     plt.rcParams['figure.figsize'] = (15,7)
#     plt.title(f'Waveform of Audio Example: {df_raw["path"]}')
#     plt.ylabel('Amplitude')

#     print(df_raw['sentence'])
#     print(f'sample_rate: {sample_rate}')
#     _ = librosa.display.waveplot(audio)
#     return ipd.Audio(audio, rate=sample_rate)

In [ ]:
timelingth_raw(raw_files_list[0])

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
len(raw_files_list)

In [ ]:
# estimation
148372 * 3/3600

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
# this is the exact calculation of the aggregated time 
# the rate is ~2 iterations in a second => it is going to take 20 hours

# total_time_length = 0
# for file in tqdm_notebook(raw_files_list):
#     total_time_length +=timelingth_raw(file)